## **Setting up dependencies**

In [3]:
%%capture
!pip install unsloth 
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [4]:
!pip install wandb bitsandbytes

In [5]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_Token")

login(token=hf_token)

In [6]:
import wandb
wb_token = user_secrets.get_secret("wandb")

wandb.login(key = wb_token)
run = wandb.init(
    project='Fine-tune-Qwen3-8B-Base on Arabic Law Dataset', 
    job_type="training", 
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohamed-tawfik-eldeeb (mohamed-tawfik-eldeeb-ta) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## **Imports**

In [7]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-07 13:42:44.607007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754574164.801666      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754574164.862797      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


## 
**Loading Model and Tokenizer**

In [8]:
model_name = "unsloth/Qwen3-8B-Base"
max_seq_length = 2048
load_in_4bit = True
dtype = None
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token, 
)

==((====))==  Unsloth 2025.8.2: Fast Qwen3 patching. Transformers: 4.55.0.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.75G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

## **Model Inference**

In [9]:
system_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following question in Arabic:

### Input:
{}

### Response:
{}"""


question = "ما هو القانون المدني ؟"

FastLanguageModel.for_inference(model)
inputs = tokenizer([system_prompt.format(question,"")] , return_tensors = "pt").to("cuda")

outputs = model.generate(
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 300,
)

response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



القانون المدني هو مجموعة من القوانين التي تنظم العلاقات بين الأفراد والمؤسسات والدولة في مجالات مختلفة مثل الملكية والعقود والمسؤولية المدنية والوراثة والزواج والطلاق. ويهدف القانون المدني إلى حماية حقوق الأفراد وضمان العدالة والمساواة في التعاملات القانونية.<|endoftext|>


In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth", 
    random_state=1024,
    use_rslora=False,  
    loftq_config=None,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)


Unsloth 2025.8.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


## **Loading and Preparing Data**

In [11]:
dataset = load_dataset("openai/MMMLU" , "AR_XY")
dataset

README.md: 0.00B [00:00, ?B/s]

mmlu_AR-XY.csv: 0.00B [00:00, ?B/s]

Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'Question', 'A', 'B', 'C', 'D', 'Answer', 'Subject'],
        num_rows: 14042
    })
})

In [12]:
df = dataset["test"].to_pandas()
df.sample(5)

,Unnamed: 0,Question,A,B,C,D,Answer,Subject
2200,135,في العام الماضي، أعدت تشيزا 32 وجبة من الحساء ...,64.0,16.0,4.0,2.0,C,elementary_mathematics
9490,82,دراسات الوبائية الرصدية مفيدة في تحديد الآثار ...,من المستحيل معرفة ما إذا كانت التأثيرات المرصو...,التعرض البشري من خلال الطعام يكون عادةً أقل بك...,تقدم الدراسات الحيوانية الخاضعة للرقابة معلوما...,يصعب الحصول على بيانات استجابة الجرعة الموثوقة...,D,nutrition
12811,374,يُفضل التصوير بالرنين المغناطيسي على التصوير ا...,الكشف عن ورم صغير,الكشف عن كسر في الجمجمة,الكشف عن احتشاء دماغي حاد,الكشف عن خلل في المادة البيضاء,B,professional_psychology
7318,34,نسبة الأطفال الذين لديهم خلل يمكن تحديده عند ا...,1 في 10,1 في 40,1 في 100,1 في 500,B,medical_genetics
12882,445,أي مما يلي يعد مثالًا لرسالة الربط المزدوج,"يقول أب لابنه: ""آمل بالتأكيد أن تتمكن من القدو...","أم تقول لابنتها: ""خطوة جيدة""، عندما تسقط الابن...","يقول المعلم لأحد الطلاب: ""يمكنك أن تفعل ذلك إذ...","يقول المعلم لأحد الطلاب: ""يمكنك أن تفعل ذلك إذ...",A,professional_psychology


In [13]:
df["Subject"].value_counts()

Subject
professional_law                       1534
moral_scenarios                         895
miscellaneous                           783
professional_psychology                 612
high_school_psychology                  545
high_school_macroeconomics              390
elementary_mathematics                  378
moral_disputes                          346
prehistory                              324
philosophy                              311
high_school_biology                     310
nutrition                               306
professional_accounting                 282
professional_medicine                   272
high_school_mathematics                 270
clinical_knowledge                      265
security_studies                        245
high_school_microeconomics              238
high_school_world_history               237
conceptual_physics                      235
marketing                               234
human_aging                             223
high_school_statistics  

In [14]:
law_data = [
    "professional_law",
    "international_law",
]

df_law = df[df["Subject"].isin(law_data)]
df_law.sample(5)

,Unnamed: 0,Question,A,B,C,D,Answer,Subject
11366,735,نشأ تقاضي في محاكم الولاية عندما حاول وزير الت...,بالاستئناف.,بأمر قضائي.,إذا صوت خمس قضاة بمراجعتها.,على الرغم من مبدأ وجود أسس كافية ومستقلة للدولة.,B,professional_law
6445,2,بعبارة أخرى، كيف يختلف مبدأ مسؤولية الحماية (R...,مبدأ مسؤولية الحماية هو نفسه التدخل الإنساني ب...,مبدأ مسؤولية الحماية يشترط طلب المساعدة من الد...,مبدأ مسؤولية الحماية أقل تعسفًا لأنه يتطلب بعض...,مبدأ مسؤولية الحماية يلجأ دائمًا إلى القوة الم...,C,international_law
10706,75,في أي من الحالات التالية تشكِّل تصرفات المدعى ...,مدّعى عليه اقتحم منزلاً عندما كان يبحث عن مأوى...,مدّعى عليه كان يمر أمام منزل فرأي صاحب المنزل ...,ذات يوم في وقت متأخر بعد الظهر، بينما كان شخصا...,ذات مساء في وقت متأخر، كان أحد النزلاء يسجّل ب...,B,professional_law
11121,490,أبلغت مريضة في مستشفى فيدرالية أن ممرضًا اعتدى...,الأدلة ضارة للغاية ويجب استبعادها.,الأدلة غير مقبولة لأنها لا علاقة لها بالتهم ال...,الأدلة مقبولة لأن السياسة العامة الحالية تشجع ...,الأدلة مقبولة بسبب التشابه الواقعي مع التهمة ا...,D,professional_law
11392,761,كان رجل يقف على ناصية أحد الشوارع عندما اقترب ...,ممنوح، لأن الإفادة الثانية كانت ثمرة الإفادة ا...,غير ممنوح، لأن الإفادة الأولى مستبعدة، والإفاد...,غير ممنوح، لأن الرجل كان قد اعترف بالفعل بالجر...,لم يتم منحها، لأن الإفادة الثانية تم الإدلاء ب...,A,professional_law


In [15]:
df_law["Response"] = df_law.apply(lambda x: x[x["Answer"]] , axis =1)
df_law.sample(4)

/tmp/ipykernel_36/2608447188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_law["Response"] = df_law.apply(lambda x: x[x["Answer"]] , axis =1)


,Unnamed: 0,Question,A,B,C,D,Answer,Subject,Response
11186,555,في صباح أحد الأيام، اتصلت امرأة هاتفيًا بجارته...,مقبولة كدليل ظرفي على أن المرأة كانت مهملة في ...,مقبولة، لأن الشاهد كانت لديه معرفة شخصية بسجل ...,مقبولة ضد الجارة كدليل على عدم لياقة المرأة.,غير مقبولة، لأن الأفعال المعينة غير مقبولة إلا...,C,professional_law,مقبولة ضد الجارة كدليل على عدم لياقة المرأة.
11301,670,قرر أحد خريجي كلية الحقوق عدم إجراء امتحان الم...,وعد واجب النفاذ يلزم العم ككفيل.,وعد غير قابل للتنفيذ، لأن والدة الخريجة كان عل...,وعد قابل للإبطال باعتباره انتهاكًا لقانون الاح...,وعد باطل منذ بدايته.,C,professional_law,وعد قابل للإبطال باعتباره انتهاكًا لقانون الاح...
10770,139,في أي من الحالات التالية لا يمكن لسُكر المدعى ...,استهلك المدعى عليه خمس قارورات من ويسكي البورب...,حضر المدعى عليه حفل زفاف في فندق، وأفرط في شرب...,ظل مدعٍ عليه بصحبة ثلاثة من رفقائه يشربون الخم...,تناول مدعٍ عليه في يوم حفل تخرجه في كلية الحقو...,D,professional_law,تناول مدعٍ عليه في يوم حفل تخرجه في كلية الحقو...
11552,921,بعد الانتظار في الطابور لمدة ساعتين للدخول إلى...,مذنبًا، لأنها كانت عمليات قتل حدثت أثناء ارتكا...,مذنبًا، لأن الإكراه ليس دفاعًا عن القتل.,غير مذنب، لأن الإكراه دفاع عن الحرق العمد.,غير مذنب، لأن المدّعى عليه كان له ما يبرره في ...,C,professional_law,غير مذنب، لأن الإكراه دفاع عن الحرق العمد.


In [16]:
df_law=df_law[['Question','Response']]
df_law.sample(5)

,Question,Response
11039,في محاولة للترويج لممارسة الجنس الآمن، بدأت مؤ...,القانون باطل لأنه ينتهك حماية التعديل الأول لح...
10725,في منتصف النهار، كان مدعى عليه يقود سيارته الر...,الاعتداء.
11093,يوفر قانون الولاية بعض التمويل للمدارس العامة ...,يجب على الطلاب أن يثبتوا أن القانون لا يرتبط ب...
11085,في صباح أحد الأيام في مغسلة، اقترب المتهم من ر...,الابتزاز والاعتداء.
11247,تقدم زوج وزوجة بطلب طلاق. القضية هي القيمة الس...,غير مقبولة، لأن الجارة لم تُثبِت أنها خبيرة في...


In [17]:
df_law.to_csv('law.csv', index=False)

In [18]:
system_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following question in Arabic:

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    outputs = examples["Response"]
    texts = []
    for input_, output_ in zip(inputs, outputs):
        text = system_prompt.format(input_, output_) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = load_dataset("csv", data_files="law.csv", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1655 [00:00<?, ? examples/s]

In [19]:
dataset['text'][7]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer the following question in Arabic:\n\n### Input:\nما الذي يعتبر "صخرة" بموجب اتفاقية الأمم المتحدة لقانون البحار (UNCLOS)؟\n\n### Response:\nالصخور هي جزر غير صالحة للسكن أو الحياة الاقتصادية الخاصة بها وبموجب المادة 121 (3) ليس لها منطقة اقتصادية خالصة أو جرف قاري<|endoftext|>'

## **Training Model**

In [20]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        output_dir="outputs",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/1655 [00:00<?, ? examples/s]

In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,655 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 43,646,976 of 8,234,382,336 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.705500
20,1.543600
30,1.519600
40,1.519000
50,1.470600
60,1.502700


In [22]:
question = "ما هو القانون المدني ؟"

FastLanguageModel.for_inference(model)
inputs = tokenizer([system_prompt.format(question,"")] , return_tensors = "pt").to("cuda")

outputs = model.generate(
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 600,
)

response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



مجموعة من القوانين التي تحدد حقوق الأفراد فيما يتعلق بالملكية والمعاملات والمسؤوليات المدنية.<|endoftext|>


## **Saving model and pushing to hugging face**

In [23]:
model.save_pretrained("arabic-law-lora")  # Save only adapter
tokenizer.save_pretrained("arabic-law-lora")

('arabic-law-lora/tokenizer_config.json',
 'arabic-law-lora/special_tokens_map.json',
 'arabic-law-lora/vocab.json',
 'arabic-law-lora/merges.txt',
 'arabic-law-lora/added_tokens.json',
 'arabic-law-lora/tokenizer.json')

In [24]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_folder(
    folder_path="arabic-law-lora",
    repo_id="Mohamed453/Arabic-Law-LoRA",
    repo_type="model"
)

CommitInfo(commit_url='https://huggingface.co/Mohamed453/Arabic-Law-LoRA/commit/8416f3f6a7d2bba7dceb81205362a84f98d44fd8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='8416f3f6a7d2bba7dceb81205362a84f98d44fd8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Mohamed453/Arabic-Law-LoRA', endpoint='https://huggingface.co', repo_type='model', repo_id='Mohamed453/Arabic-Law-LoRA'), pr_revision=None, pr_num=None)

In [33]:
new_model_online = "Mohamed453/Arabic-Law-Meta-Qwen3-8B-Base"
new_model_local = "Arabic-Law-Meta-Qwen3-8B-Base"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

('Arabic-Law-Meta-Qwen3-8B-Base/tokenizer_config.json',
 'Arabic-Law-Meta-Qwen3-8B-Base/special_tokens_map.json',
 'Arabic-Law-Meta-Qwen3-8B-Base/vocab.json',
 'Arabic-Law-Meta-Qwen3-8B-Base/merges.txt',
 'Arabic-Law-Meta-Qwen3-8B-Base/added_tokens.json',
 'Arabic-Law-Meta-Qwen3-8B-Base/tokenizer.json')

In [37]:
model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged(new_model_online, tokenizer, save_method = "merged_16bit")

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  75%|███████▌  | 3/4 [11:22<03:45, 225.74s/it]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [12:29<00:00, 187.43s/it]
